In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')

    from google.colab import drive
    drive.mount('/content/drive')   

    import os
    os.chdir('/content/drive/My Drive/Deep_learning_project/')

else:
    print('Not running on CoLab')

Running on CoLab
Mounted at /content/drive


# LOAD AND PREPROCESS RAW DATA

In [ ]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import pickle
from torch.optim import lr_scheduler
from torch.distributions import Normal
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

from modules import RealNVP

cuda = torch.cuda.is_available()
device = "cuda:0" if cuda else "cpu"

In [ ]:
df = pd.read_csv('GM_preparedData.csv')

In [ ]:
# Define rectangle where "valid data belongs":

upper_right = (55.880120, 12.707644)
lower_left = (55.548626, 12.061736)

# Clean data
df_clean = df.loc[(df['startPositionLat'] >= lower_left[0]) & (df['startPositionLat']<=upper_right[0])]
df_clean = df_clean.loc[(df['startPositionLng'] >= lower_left[1]) & (df_clean['startPositionLng']<=upper_right[1])]

df_clean = df_clean.dropna(axis=0, subset=['startPositionLat', 'startPositionLat'])

In [ ]:
data_columns = ['startPositionLng', 'startPositionLat']
data = df_clean[data_columns]

In [ ]:
train, test = train_test_split(data.values, test_size=0.1, random_state=42)
df_train = pd.DataFrame(train, columns=data.columns)
df_test = pd.DataFrame(test, columns=data.columns)

#Normalize training data
mu_lat_start = df_train['startPositionLat'].mean()
sig_lat_start = df_train['startPositionLat'].std()

mu_lng_start = df_train['startPositionLng'].mean()
sig_lng_start = df_train['startPositionLng'].std()

df_train['NormLngStart'] = df_train['startPositionLng'].apply(lambda x: (x-mu_lng_start)/sig_lng_start)
df_train['NormLatStart'] = df_train['startPositionLat'].apply(lambda x: (x-mu_lat_start)/sig_lat_start)

# Normalize validation set
test_mu_lat_start = df_test['startPositionLat'].mean()
test_sig_lat_start = df_test['startPositionLat'].std()

test_mu_lng_start = df_test['startPositionLng'].mean()
test_sig_lng_start = df_test['startPositionLng'].std()

df_test['NormLngStart'] = df_test['startPositionLng'].apply(lambda x: (x-test_mu_lng_start)/test_sig_lng_start)
df_test['NormLatStart'] = df_test['startPositionLat'].apply(lambda x: (x-test_mu_lat_start)/test_sig_lat_start)


In [ ]:
data_train = df_train[['NormLngStart', 'NormLatStart']]
data_test = df_test[['NormLngStart', 'NormLatStart']]

# Vanilla Normalizing Flow

In [ ]:
# Define the base distribution and its parameters
mu = torch.zeros(2).to(device)
sigma = torch.ones(2).to(device)
base = Normal(mu, sigma)

# The RealNVP model is a stack of identical coupling layers, here we use 6 couplng layers.
net = RealNVP(in_features=2, prior=base, hidden_features=256, depth=6).to(device)
optimizer = torch.optim.Adam([p for p in net.parameters() if p.requires_grad], lr=1e-4)

pyro_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=25, verbose=True)

# Arbitrary clipping value to suppress exploding gradients
clipping_value = 1

In [ ]:
X_train = torch.Tensor(data_train.values).to(device)
X_test = torch.Tensor(data_test.values).to(device)

train_dataset = TensorDataset(X_train)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [ ]:
train_losses = []
validation_losses = []
save_flag = True
epochs = 500

try:
    for epoch in tqdm(range(epochs)):
        loss_epoch = 0

        # VALIDATION
        net.eval()

        validation_loss = -torch.mean(net.log_likelihood(X_test))
        validation_losses.append(validation_loss.item())
        
        pyro_scheduler.step(validation_loss)

        # TRAIN LOOP
        net.train()
        for batch_num, (X_batch) in enumerate(train_loader):
            X_batch = X_batch[0]
            
            optimizer.zero_grad()
            
            loss = -torch.mean(net.log_likelihood(X_batch))
            
            loss.backward(retain_graph=True)
            torch.nn.utils.clip_grad_norm_(net.parameters(), clipping_value)
            optimizer.step()

            loss_epoch += loss.item()

        loss_epoch = loss_epoch / (batch_num+1)
        train_losses.append(loss_epoch)

        if (epoch) % 1 == 0:
            print(f'Validation loss: {validation_loss}')
            print(f'Train loss: {loss_epoch}')

        if (epoch) % 50 == 0:
            torch.save(net.state_dict(), 'model_state_vanilla_NF_updated_epoch_%d.pth' % (epoch+1))
    
    if save_flag:
        loss_dict = {'validation': validation_losses, 'train': train_losses}
        with open('loss_dict.pkl', 'wb') as f:
            pickle.dump(loss_dict, f)

        torch.save(net.state_dict(), 'model_state_vanilla_NF_updated.pth')

except KeyboardInterrupt:
    if save_flag:
        loss_dict = {'validation': validation_losses, 'train': train_losses}
        with open('loss_dict.pkl', 'wb') as f:
            pickle.dump(loss_dict, f)

        torch.save(net.state_dict(), 'model_state_vanilla_NF_updated.pth')

  0%|          | 1/500 [00:19<2:39:29, 19.18s/it]

Validation loss: 1.3312300443649292
Train loss: 0.9397779525170148


  0%|          | 2/500 [00:38<2:38:26, 19.09s/it]

Validation loss: 0.9295644164085388
Train loss: 0.8919068588713421


  1%|          | 3/500 [00:56<2:37:38, 19.03s/it]

...

Validation loss: 0.4606465995311737
Train loss: 0.43282729778031276


100%|█████████▉| 499/500 [2:30:41<00:20, 20.03s/it]

Validation loss: 0.46096986532211304
Train loss: 0.43293337132709825


100%|██████████| 500/500 [2:31:01<00:00, 18.12s/it]

Validation loss: 0.4607619047164917
Train loss: 0.4329096090379369
